# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

OpenAI API Key exists and begins sk-proj-


## Adding more tools

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

seats_available = {"london": "100", "paris": "80", "tokyo": "70", "berlin": "50"}

def get_availability(destination_city):
    print(f"Tool get_availbility checking for {destination_city}")
    city = destination_city.lower()
    return seats_available.get(city, "Unknown")

booking_function = {
    "name": "get_availability",
    "description": "Get the number of available seats of a ticket to the destination city. Call this whenever you need to know the availability of ticket, for example when a customer asks 'availability of tickets'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}



tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}]


In [ ]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_Ztool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

# We have to write that function handle_tool_call:
# def handle_tool_call(message):
#     tool_call = message.tool_calls[0]
#     arguments = json.loads(tool_call.function.arguments)
#     city = arguments.get('destination_city')
#     price = get_ticket_price(city)
#     availability = get_availability(city)
#     response = {
#         "role": "tool",
#         "content": json.dumps({"destination_city": city,"price": price,"availability": availability}),
#         "tool_call_id": tool_call.id
#     }
#     return response, city

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_calls(message)

        messages.append(message)
        messages.extend(tool_responses)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

def handle_tool_calls(message):
    tool_responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')

        if tool_call.function.name == "get_ticket_price":
            result = get_ticket_price(city)
            content = json.dumps({"destination_city": city, "price": result})
        elif tool_call.function.name == "get_availability":
            result = get_availability(city)
            content = json.dumps({"destination_city": city, "availability": result})
        else:
            content = json.dumps({"error": f"Unknown tool: {tool_call.function.name}"})

        tool_responses.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": content
        })

    return tool_responses

In [ ]:
# gr.ChatInterface(fn=chat, type="messages").launch() #this is working well so far

## Adding another Agent

In [6]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."


ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
seats_available = {"london": "100", "paris": "80", "tokyo": "70", "berlin": "50"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}


def get_availability(destination_city):
    print(f"Tool get_availbility checking for {destination_city}")
    available= destination_city.lower()
    return seats_available.get(available, "Unknown")

                       
booking_function = {
    "name" : "get_availability",
    "description": "Get the availability of flight seat to the destination city. Call this whenever you need to know the ticket price, for exmaple, when a customer asks 'I want to book flight ticket.'",
    "parameters": {
        "type":"object",
        "properties":{
            "destination_city":{
                "type":"string",
                "description":"The city that the customer wants to travel to.",
            },
        },
        "required":["destination_city"],
        "additionalProperties":False
    }
}


tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}]



def handle_tool_calls(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    name = tool_call.function.name
    city = arguments.get('destination_city')
    

    if name.replace('"','') == "get_ticket_price":
        price = get_ticket_price(city)
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
        }

    elif name.replace('"','') == "get_availability":
        availability = get_availability(city)
        response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"availability": availability},),
        "tool_call_id": tool_call.id
        }


    return response, city




def chat(history):
    if history is None:
        history = []
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools) # tool_choice="auto" added
    # info_text = "" # <-> image = None

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        name = json.dumps(tool_call.function.name)
        city = arguments.get('destination_city')
        if name.replace('"','') == "get_availability":
                availability = get_availability(city)
                response = {
                                "role": "tool",
                                "content": json.dumps({"destination_city": city, "availability": availability}),
                                "tool_call_id": tool_call.id
                            }
                messages.append(message)
                messages.append(response)
                print(messages)
                response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

        else:    
            response, city = handle_tool_calls(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
           
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]    

    return history





# def info(city):
#     messages =  [{"role": "system", "content": f"Information about the {city} for tourists, kind of today's currency, weather, tourists attraction, and so on."}] 
#     info_response = openai.chat.completions.create(model=MODEL, messages = messages)
#     return info_response.choices[0].message.content


def info(city):
    messages =  [{
        "role": "system",
        "content": f"Information about the {city} for tourists, kind of today's currency, weather, tourists attraction, and so on."
    }] 
    info_response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return info_response.choices[0].message.content


with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")

    with gr.Row():
        output = gr.Markdown(label="Tourist Info")

    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history


    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot]
    ).then(fn=info, inputs= entry, outputs = output)

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)


ui.launch()

OpenAI API Key exists and begins sk-proj-


ERROR:    [WinError 10014] 호출에 대한 포인터 인수를 사용하려는 동안 시스템에서 잘못된 포인터 주소를 감지했습니다
Exception in thread Thread-26 (run):
Traceback (most recent call last):
  File "C:\Users\82103\anaconda3\envs\llms\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\82103\anaconda3\envs\llms\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\82103\anaconda3\envs\llms\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\82103\anaconda3\envs\llms\Lib\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\82103\anaconda3\envs\llms\Lib\asyncio\runners.py", line 189, in run
    with Runner(debug=debug) as runner:
  File "C:\Users\82103\anaconda3\envs\llms\Lib\asyncio\runners.py", line 59, in __enter__
    self._lazy_init()
  File "C:\Users\82103\anaconda3\envs\l

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Tool get_availbility checking for Tokyo
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."}, {'role': 'user', 'metadata': None, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'metadata': None, 'content': "I'd like to go to the Tokyo", 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Would you like to know the ticket price or flight availability to Tokyo?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'availability', 'options': None}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_h5jv4PbkUIwSz4PHUpWXWZLj', function=Function(arguments='{"destination_city":"Tokyo"}